In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import ResNet50_Weights
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [17]:
train_dataset_path = 'dataset/train'
test_dataset_path = 'dataset/test'
gop3_dataset_path = "dataset/gop3_dataset"

In [18]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (or any size that fits the model)
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize based on ImageNet stats
])

In [19]:
train_card_dataset = datasets.ImageFolder(root=train_dataset_path, transform=transform)
test_card_dataset = datasets.ImageFolder(root=test_dataset_path, transform=transform)
gop3_card_dataset = datasets.ImageFolder(root=gop3_dataset_path, transform=transform)

In [28]:
batch_size = 48  # Set the batch size according to your GPU memory or system capacity
train_card_dataloader = DataLoader(train_card_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_card_dataloader = DataLoader(test_card_dataset, batch_size=batch_size, shuffle=False, num_workers=8)
gop3_card_dataloader = DataLoader(gop3_card_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [29]:
print("Class to Index Mapping:")
print(gop3_card_dataset.class_to_idx)

Class to Index Mapping:
{'2c': 0, '2d': 1, '2h': 2, '2s': 3, '3c': 4, '3d': 5, '3h': 6, '3s': 7, '4c': 8, '4d': 9, '4h': 10, '4s': 11, '5c': 12, '5d': 13, '5h': 14, '5s': 15, '6c': 16, '6d': 17, '6h': 18, '6s': 19, '7c': 20, '7d': 21, '7h': 22, '7s': 23, '8c': 24, '8d': 25, '8h': 26, '8s': 27, '9c': 28, '9d': 29, '9h': 30, '9s': 31, 'Ac': 32, 'Ad': 33, 'Ah': 34, 'As': 35, 'Jc': 36, 'Jd': 37, 'Jh': 38, 'Js': 39, 'Kc': 40, 'Kd': 41, 'Kh': 42, 'Ks': 43, 'Qc': 44, 'Qd': 45, 'Qh': 46, 'Qs': 47, 'Tc': 48, 'Td': 49, 'Th': 50, 'Ts': 51}


In [9]:
len(train_card_dataset.classes)

52

In [13]:
# Load the pretrained ResNet50 model
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Freeze all layers except the last fully connected layer
# for param in model.parameters():
#     param.requires_grad = False

# Replace the final layer with a new layer with the appropriate number of output classes
num_classes = len(train_card_dataset.classes)  # Number of classes (52 for a deck of cards, for example)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to the device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [33]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1)  # Only the parameters of the final layer

In [34]:
# Training parameters
num_epochs = 10  # Set the number of epochs
best_accuracy = 0.0

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate over batches
    for images, labels in tqdm(gop3_card_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy metrics
        running_loss += loss.item() * images.size(0)  # Sum loss for the batch
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_card_dataset)
    epoch_accuracy = correct / total * 100

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Save the model if it's the best accuracy achieved
    if epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        torch.save(model.state_dict(), 'best_card_model.pth')
        print("Model saved as best_card_model.pth")


Epoch 1/10: 100%|██████████| 2/2 [00:03<00:00,  1.63s/it]


Epoch [1/10], Loss: 0.1899, Accuracy: 3.85%
Model saved as best_card_model.pth


Epoch 2/10: 100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


Epoch [2/10], Loss: 0.6253, Accuracy: 0.00%


Epoch 3/10: 100%|██████████| 2/2 [00:03<00:00,  1.65s/it]


...

Epoch [6/10], Loss: 0.3049, Accuracy: 0.00%


Epoch 7/10: 100%|██████████| 2/2 [00:03<00:00,  1.69s/it]


Epoch [7/10], Loss: 0.3063, Accuracy: 3.85%


Epoch 8/10:   0%|          | 0/2 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [30]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import torch

# Load the best model
model.load_state_dict(torch.load('best_card_model.pth'))
model.eval()  # Set the model to evaluation mode

# Initialize lists to store true and predicted labels
all_labels = []
all_preds = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for images, labels in tqdm(gop3_card_dataloader, desc="Test"):
        images, labels = images.to(device), labels.to(device)

        # Get predictions
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Store labels and predictions for metrics calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate overall accuracy
accuracy = accuracy_score(all_labels, all_preds)

# Calculate precision, recall, and F1 score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(all_labels, all_preds, average=None, labels=range(num_classes))

# Calculate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Print overall accuracy
print(f"Overall Accuracy: {accuracy * 100:.2f}%\n")

# Print precision, recall, and F1 score for each class
print("Class-wise Precision, Recall, and F1 Score:")
for idx, class_name in enumerate(train_card_dataset.classes):
    print(f"{class_name}: Precision: {precision[idx]:.2f}, Recall: {recall[idx]:.2f}, F1 Score: {f1_score[idx]:.2f}")

# Print confusion matrix
print("\nConfusion Matrix:")
print(conf_matrix)

Test: 100%|██████████| 2/2 [00:03<00:00,  1.73s/it]

Overall Accuracy: 13.46%

Class-wise Precision, Recall, and F1 Score:
2c: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
2d: Precision: 0.10, Recall: 1.00, F1 Score: 0.18
2h: Precision: 1.00, Recall: 1.00, F1 Score: 1.00
2s: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
3c: Precision: 0.50, Recall: 1.00, F1 Score: 0.67
3d: Precision: 1.00, Recall: 1.00, F1 Score: 1.00
3h: Precision: 0.50, Recall: 1.00, F1 Score: 0.67
3s: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
4c: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
4d: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
4h: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
4s: Precision: 0.25, Recall: 1.00, F1 Score: 0.40
5c: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
5d: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
5h: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
5s: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
6c: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
6d: Precision: 0.00, Recall: 0.00, F1 Score: 0.00
6h: Precision: 0.00, Recall: 0